In [71]:
import struct


def ieee745_float_to_hex(f):
    return hex(struct.unpack("<I", struct.pack("<f", f))[0])


def ieee745_float_to_bin(num):
    (bits,) = struct.unpack("!I", struct.pack("!f", num))
    return "{:032b}".format(bits)


def ieee745_float_to_int(num):
    (bits,) = struct.unpack("!I", struct.pack("!f", num))
    return bits


def ieee745_bin_to_float(N):  # ieee-745 bits (max 64 bit)
    a = int(N[0])  # sign,     1 bit
    b = int(N[1:12], 2)  # exponent, 8 bits
    c = int("1" + N[12:], 2)  # fraction, len(N)-9 bits

    return (-1) ** a * c / (1 << (len(N) - 9 - (b - 1024)))


def int_to_bin(num):

    return "{0:b}".format(num)


def bin_to_int(num):

    return int(num, 2)


def hex_to_int(num):

    return int(num, 16)


import ctypes


def float_to_int(num):
    
    return ctypes.c_uint32.from_buffer(ctypes.c_float(num)).value


def int_to_float(num):

    return ctypes.c_float.from_buffer(ctypes.c_uint32(num)).value


In [89]:
import numpy as np

THREEHALFS = np.float32(1.5)
L = 2 ** 23
B = 127
SIG = 0.0430357
SHIFT = int(round(3 / 2 * L * (B - SIG)))
SHIFT = int("0x5f3759df", 16)


def rsqrt(a):

    a = np.float32(a)

    # evil floating point bit level hacking
    i = float_to_int(a)
    # what the fuck?
    i = SHIFT - (i >> 1)
    y = int_to_float(i)

    x2 = np.float32(0.5) * a
    # 1st iteration
    y = y * (THREEHALFS - (x2 * y * y))
    # 2nd iteration, this can be removed
    y = y * (THREEHALFS - (x2 * y * y))

    return y


import timeit

print(rsqrt(2.0))
print(1 / np.sqrt(2.0))

# print(timeit.timeit("rsqrt(2.0)", globals=globals()))
# print(timeit.timeit("1/np.sqrt(2.0)", setup="import numpy as np"))


0.7071067149264607
0.7071067811865475
